### Knight_board module

In [1]:
import math
import random


class Board:
    """
    board_size (int): size of the board
    knight_pos (tuple of int): position of the knight
    destination (tuple of int): position of the goal
    coin_pos_set (set of tuple): set of position of the coins
    obstacle_pos_set (set of tuple): set of position of the obstacles
    """

    def __init__(self, board_size, knight_pos, destination, coin_pos_set, obstacle_pos_set, algorithm):
        self.board_size = board_size
        self.knight_pos = knight_pos
        self.destination = destination
        self.coin_pos_set = coin_pos_set
        self.obstacle_pos_set = obstacle_pos_set
        self.coin_value = -3
        self.normal_value = 1
        if algorithm==None:
          algorithm= input("Please choose algorithm: 'UCS' or 'Astar' or 'backtrack'")
        self.algorithm = algorithm
        if self.algorithm== 'UCS':
          self.goal_value = 3 #math.ceil(board_size*2/3)
        elif self.algorithm== 'Astar':
          self.goal_value = 3
        elif self.algorithm== 'backtrack':
          self.goal_value= 0

    def possible_moves(self):
        """
        return the list of possible next positions of the knight
        """
        x, y = self.knight_pos
        possible_moves = []

        if x - 2 >= 0 and y - 1 >= 0:
            possible_moves.append((x - 2, y - 1))

        if x - 2 >= 0 and y + 1 < self.board_size:
            possible_moves.append((x - 2, y + 1))

        if x - 1 >= 0 and y - 2 >= 0:
            possible_moves.append((x - 1, y - 2))

        if x - 1 >= 0 and y + 2 < self.board_size:
            possible_moves.append((x - 1, y + 2))

        if x + 1 < self.board_size and y - 2 >= 0:
            possible_moves.append((x + 1, y - 2))

        if x + 1 < self.board_size and y + 2 < self.board_size:
            possible_moves.append((x + 1, y + 2))

        if x + 2 < self.board_size and y - 1 >= 0:
            possible_moves.append((x + 2, y - 1))

        if x + 2 < self.board_size and y + 1 < self.board_size:
            possible_moves.append((x + 2, y + 1))

        for i in range(len(possible_moves) - 1, -1, -1):
            if possible_moves[i] in self.obstacle_pos_set:
                possible_moves.pop(i)
        return possible_moves

    def move_knight(self, new_pos):
        """
        Update self.knight_pos to new_pos
        Remove coin from set if knight jump to the square that has coin

        Return the new board and the value of the square that the knight jump to
        """
        assert new_pos in self.possible_moves(), f"The knight can't go to {new_pos}"

        new_coin_set = self.coin_pos_set.copy()
        if new_pos in self.coin_pos_set:
            new_coin_set.remove(new_pos)
            value = self.coin_value
        elif new_pos == self.destination:
            value = self.goal_value
        else:
            value = self.normal_value

        board = Board(self.board_size, new_pos, self.destination, new_coin_set,
                      self.obstacle_pos_set,self.algorithm)

        return board, value

### Problem module

In [2]:
class Problem:
    """
    board_size (int): size of the board
    num_coin (int): number of coins on board
    num_obstacles (int): number of obstacles on board
    knight_pos (tuple): optional, initial position of the knight
    dest_pos (tuple): optional, the position of the goal
    """

    def __init__(self, board_size, num_coin, num_obstacles, coin_pos_set=None, obstacle_pos_set=None, knight_pos=None,
                 dest_pos=None,algorithm=None):
        self.board_size = board_size
        self.num_coin = num_coin
        self.num_obstacles = num_obstacles
        self.coin_pos_set = coin_pos_set
        self.obstacle_pos_set = obstacle_pos_set
        self.knight_pos = knight_pos
        self.dest_pos = dest_pos
        self.algorithm=algorithm

        if self.knight_pos == None:
            self.knight_pos = self.generate_knight_pos()
        if self.dest_pos == None:
            self.dest_pos = self.generate_dest_pos()
        if self.coin_pos_set == None:
            self.coin_pos_set = self.generate_coin()
        if self.obstacle_pos_set == None:
            self.obstacle_pos_set = self.generate_obstacle()

    def initial_state(self):
        """
        Return the initial state of the board
        """

        self.board = Board(self.board_size, self.knight_pos, self.dest_pos, self.coin_pos_set,
                           self.obstacle_pos_set,self.algorithm)

        return self.board

    def goal_test(self, state):
        """
        Check if the state is the goal
        """
        if state.knight_pos == state.destination:
            return True
        return False

    def actions(self, state):
        """
        return possible next actions from current state
        """
        possible_moves = state.possible_moves()
        return possible_moves

    def generate_knight_pos(self):
        """
        board_size (int): size of the board

        Return: (tuple) position of the knight
        """
        x = random.randint(0, self.board_size - 1)
        y = random.randint(0, self.board_size - 1)
        return (x, y)

    def generate_dest_pos(self):
        """
        board_size (int): size of the board
        knight_pos (tuple): position of the knight

        Return: position of the goal
        """
        x, y = self.knight_pos
        while (x, y) == self.knight_pos:
            x = random.randint(0, self.board_size - 1)
            y = random.randint(0, self.board_size - 1)
        return (x, y)

    def generate_coin(self):
        """
        num_coin (int): number of coin on the board
        board_size (int): size of the board
        knight_pos (tuple): position of the knight
        dest_pos (tuple): position of the goal

        return: coin_pos_set, a set of all positions of the coins 
        """
        coin_pos_set = set()
        while len(coin_pos_set) < self.num_coin:
            x = random.randint(0, self.board_size - 1)
            y = random.randint(0, self.board_size - 1)
            if (x, y) != self.knight_pos and (x, y) != self.dest_pos:
                coin_pos_set.add((x, y))

        return coin_pos_set

    def generate_obstacle(self):
        """
        num_obstacles (int): number of obstacles on the board
        board_size (int): size of the board
        knight_pos (tuple): position of the knight
        dest_pos (tuple): position of the goal
        coin_pos_set (set of tuples): set of position of coins

        return: obstacle_pos_set, set of all positions of obstacles
        """
        obstacle_pos_set = set()
        while len(obstacle_pos_set) < self.num_obstacles:
            x = random.randint(0, self.board_size - 1)
            y = random.randint(0, self.board_size - 1)
            if (x, y) not in self.coin_pos_set and (x, y) != self.knight_pos and (x, y) != self.dest_pos:
                obstacle_pos_set.add((x, y))
        return obstacle_pos_set


### Visual_demonstrate module

In [3]:
from tkinter import *


def show_board(board_size, knight_pos, coin_set, obs_set, destination, path=[], coin_value=0, normal_value=0):
    print('Rule: Coin +{}, Empty squares {}'.format(-coin_value, -normal_value))
    coin_value = -coin_value
    # window
    window = Tk()
    window.title('Simulation')
    Grid.rowconfigure(window, 0, weight=1)
    Grid.columnconfigure(window, 0, weight=1)
    size = str(int((board_size + 0.5) * 80)) + 'x' + str(int((board_size + 1.5) * 80))
    window.geometry(size)
    square = {}
    window = Frame(master=window)
    w_knight_image = PhotoImage(file="images/white_knight.png")
    b_knight_image = PhotoImage(file="images/black_knight.png")
    w_mount_img = PhotoImage(file='images/white_mountain.png')
    b_mount_img = PhotoImage(file='images/black_mountain.png')
    w_coin_img = PhotoImage(file='images/white_coin.png')
    b_coin_img = PhotoImage(file='images/black_coin.png')
    w_flag_img = PhotoImage(file='images/white_flag.png')
    b_flag_img = PhotoImage(file='images/black_flag.png')
    Grid.rowconfigure(window, 0, weight=1)
    Grid.columnconfigure(window, 0, weight=1)

    board_frame = Frame(master=window, relief='raised', border=10)

    board_frame.grid(row=0, column=0, sticky=N + S + E + W)
    for i in range(0, board_size + 1):
        if i == 0:
            Grid.columnconfigure(board_frame, i, weight=1, uniform='column')
            Grid.rowconfigure(board_frame, i, weight=1, uniform='row')
        elif i == board_size:
            Grid.columnconfigure(board_frame, i, weight=2, uniform='column')
            Grid.rowconfigure(board_frame, i, weight=2, uniform='row')
        else:
            Grid.columnconfigure(board_frame, i, weight=2, uniform='column')
            Grid.rowconfigure(board_frame, i, weight=2, uniform='row')

        for j in range(0, board_size + 1):
            if i * j == 0:
                if i != 0:
                    row_label = Label(board_frame, text=str(i - 1), font=('arial', 18, 'bold'))
                    row_label.grid(row=i, column=j)
                elif j != 0:
                    col_label = Label(board_frame, text=str(j - 1), font=('arial', 18, 'bold'))
                    col_label.grid(row=i, column=j)
            else:
                if (i % 2) - (j % 2) == 0:
                    square[i, j] = [Frame(board_frame, relief='ridge', border=2, bg='white')]
                    if (i - 1, j - 1) == knight_pos:
                        square[i, j].append(Label(square[i, j][0], image=w_knight_image, bg='white'))
                        Grid.rowconfigure(square[i, j][0], 0, weight=1)
                        Grid.columnconfigure(square[i, j][0], 0, weight=1)
                        square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                    elif (i - 1, j - 1) in obs_set:
                        square[i, j].append(Label(square[i, j][0], image=w_mount_img, bg='white'))
                        Grid.rowconfigure(square[i, j][0], 0, weight=1)
                        Grid.columnconfigure(square[i, j][0], 0, weight=1)
                        square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                    elif (i - 1, j - 1) in coin_set:
                        square[i, j].append(Label(square[i, j][0], image=w_coin_img, bg='white'))
                        Grid.rowconfigure(square[i, j][0], 0, weight=1)
                        Grid.columnconfigure(square[i, j][0], 0, weight=1)
                        square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                    elif (i - 1, j - 1) == destination:
                        square[i, j].append(Label(square[i, j][0], image=w_flag_img, bg='white'))
                        Grid.rowconfigure(square[i, j][0], 0, weight=1)
                        Grid.columnconfigure(square[i, j][0], 0, weight=1)
                        square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                    square[i, j][0].grid(row=i, column=j, sticky=N + S + E + W)

                else:
                    square[i, j] = [Frame(board_frame, bg='black')]
                    if (i - 1, j - 1) == knight_pos:
                        square[i, j].append(Label(square[i, j][0], image=b_knight_image, bg='black'))
                        Grid.rowconfigure(square[i, j][0], 0, weight=1)
                        Grid.columnconfigure(square[i, j][0], 0, weight=1)
                        square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                    elif (i - 1, j - 1) in obs_set:
                        square[i, j].append(Label(square[i, j][0], image=b_mount_img, bg='black'))
                        Grid.rowconfigure(square[i, j][0], 0, weight=1)
                        Grid.columnconfigure(square[i, j][0], 0, weight=1)
                        square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                    elif (i - 1, j - 1) in coin_set:
                        square[i, j].append(Label(square[i, j][0], image=b_coin_img, bg='black'))
                        Grid.rowconfigure(square[i, j][0], 0, weight=1)
                        Grid.columnconfigure(square[i, j][0], 0, weight=1)
                        square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                    elif (i - 1, j - 1) == destination:
                        square[i, j].append(Label(square[i, j][0], image=b_flag_img, bg='black'))
                        Grid.rowconfigure(square[i, j][0], 0, weight=1)
                        Grid.columnconfigure(square[i, j][0], 0, weight=1)
                        square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                    square[i, j][0].grid(row=i, column=j, sticky=N + S + E + W)
    point = 0
    l = 0

    def next_move():
        nonlocal l, point
        if l >= 0 and l < (len(path) - 1):
            (prev_i, prev_j) = path[l][0] + 1, path[l][1] + 1
            (i, j) = path[l + 1][0] + 1, path[l + 1][1] + 1
            square[prev_i, prev_j][1].grid_forget()
            square[prev_i, prev_j].pop()
            if len(square[i, j]) > 1:
                square[i, j][1].grid_forget()
                square[i, j].pop()
                if (i - 1, j - 1) in coin_set:
                    point += coin_value
            else:
                point -= 1
            point_text = 'Point = ' + str(point)
            obj_label.config(text=point_text)
            if (i % 2) - (j % 2) == 0:
                square[i, j].append(Label(square[i, j][0], image=w_knight_image, bg='white'))
            else:
                square[i, j].append(Label(square[i, j][0], image=b_knight_image, bg='black'))

            square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
            l += 1
        else:
            pass

    def prev_move():
        nonlocal l, point
        if l > 0 and l < len(path):
            (i, j) = path[l][0] + 1, path[l][1] + 1
            (prev_i, prev_j) = path[l - 1][0] + 1, path[l - 1][1] + 1
            square[i, j][1].grid_forget()
            square[i, j].pop()
            if (i - 1, j - 1) in coin_set and (i - 1, j - 1) not in path[:l]:
                if (i % 2) - (j % 2) == 0:

                    square[i, j].append(Label(square[i, j][0], image=w_coin_img, bg='white'))
                    square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                else:
                    square[i, j].append(Label(square[i, j][0], image=b_coin_img, bg='black'))
                    square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                point -= coin_value
            elif (i - 1, j - 1) == destination:
                if (i % 2) - (j % 2) == 0:
                    square[i, j].append(Label(square[i, j][0], image=w_flag_img, bg='white'))
                    square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
                else:
                    square[i, j].append(Label(square[i, j][0], image=b_flag_img, bg='black'))
                    square[i, j][1].grid(row=0, column=0, sticky=N + S + E + W)
            else:
                point += 1
            point_text = 'Point = ' + str(point)
            obj_label.config(text=point_text)
            if (prev_i % 2) - (prev_j % 2) == 0:
                square[prev_i, prev_j].append(Label(square[prev_i, prev_j][0], image=w_knight_image, bg='white'))
            else:
                square[prev_i, prev_j].append(Label(square[prev_i, prev_j][0], image=b_knight_image, bg='black'))

            square[prev_i, prev_j][1].grid(row=0, column=0, sticky=N + S + E + W)
            l -= 1
        else:
            pass

    obj_frame = Frame(master=window, relief='flat', border=5)
    point_text = 'Point = ' + str(0)
    obj_label = Label(master=obj_frame, text=point_text, font=('arial', 18, 'bold'))
    next_move_button = Button(obj_frame, text='Next move', font=('arial', 12, 'bold'), relief='raised', border=4,
                              command=next_move)
    next_move_button.pack(side=RIGHT)
    prev_move_button = Button(obj_frame, text='Prev move', font=('arial', 12, 'bold'), relief='raised', border=4,
                              command=prev_move)
    prev_move_button.pack(side=RIGHT)
    obj_label.pack()
    obj_frame.grid(row=board_size, columnspan=board_size, sticky=N + S + E + W)
    window.grid(row=0, column=0)
    window.lift()
    window.mainloop()


### UCS

In [4]:
from queue import PriorityQueue


class Node:
    def __init__(self, board):
        self.state = board
        self.parent = None


def child_node(node, action):
    child, gain = node.state.move_knight(action)
    child = Node(child)
    child.parent = node
    return child, gain


def solution(node, point):
    print('Solution found!')
    print('Objective:', -point + node.state.goal_value)
    print('Path:')
    path = []
    path.append(node.state.knight_pos)
    while node.parent != None:
        node = node.parent
        path.append(node.state.knight_pos)
    print(path[::-1])
    path = path[::-1]
    return path, -point + node.state.goal_value


def hash(state):
    hashkey = ''
    hashkey += (str(state.knight_pos[0]) + str(state.knight_pos[1]))
    for item in state.coin_pos_set:
        hashkey += (str(item[0]) + str(item[1]))
    return hashkey


def uniform_cost_search(problem):
    print('\nFinding solution...')
    node = Node(problem.initial_state())
    point = 0
    if problem.goal_test(node.state):
        return solution(node, point)
    queue = PriorityQueue()
    queue.put((point, id(node), node))
    frontier = {}
    frontier[hash(node.state)] = 0
    explored = {}
    parent = {}
    parent[hash(node.state)] = None

    while not queue.empty():
        dequeued_obj = queue.get()
        node = dequeued_obj[2]
        node_key = hash(node.state)
        try:
            del frontier[node_key]
            point = dequeued_obj[0]
            if problem.goal_test(node.state):
                return solution(node, point)
            explored[node_key] = point
            for action in problem.actions(node.state):
                child, gain = child_node(node, action)
                new_point = point
                new_point += gain
                state = hash(child.state)

                if state not in explored and state not in frontier:
                    queue.put((new_point, id(child), child))
                    frontier[state] = new_point
                elif state in explored and new_point < explored[state]:
                    explored[state] = point
                    queue.put((new_point, id(child), child))
                    frontier[state] = new_point
                elif state in frontier and new_point < frontier[state]:
                    queue.put((new_point, id(child), child))
                    frontier[state] = new_point
        except KeyError:
            pass
    print('INFEASIBLE')
    return [],0


In [5]:
from time import perf_counter

if __name__ == "__main__":
    board_size = 8  # int(input('Board size: '))
    num_coin = 9  # int(input('Number of coins: '))
    num_obs = 8  # int(input('Number of obstacles: '))
    coin_pos_set = {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}
    random.seed(2)
    print('Generating environment...')

    problem = Problem(board_size=board_size, num_coin=num_coin, num_obstacles=num_obs, algorithm='UCS')
    print(
        'Environment:\nBoard_size:', problem.board_size, '\nNumber of coins:', problem.num_coin,
        '\nCoin positions set:',
        problem.coin_pos_set, '\nNumber of obstacle:', problem.num_obstacles, '\nObstacles\' set:',
        problem.obstacle_pos_set,
        '\nKnight\'s pos:', problem.knight_pos,
        '\nDestination:', problem.dest_pos, '\n')
    start = perf_counter()
    path, point = uniform_cost_search(problem)
    print('Runtime:', perf_counter() - start)
    if path != None:
        a ='n'# input('Run the demonstration of the path? [y/n] ')
        if a == 'y':
            show_board(problem.board_size, problem.knight_pos, problem.coin_pos_set, problem.obstacle_pos_set,
                       problem.dest_pos, path, problem.board.coin_value, problem.board.normal_value)


Generating environment...
Environment:
Board_size: 8 
Number of coins: 9 
Coin positions set: {(6, 6), (2, 2), (5, 7), (7, 5), (0, 5), (4, 3), (2, 4), (0, 2), (4, 0)} 
Number of obstacle: 8 
Obstacles' set: {(2, 7), (6, 7), (3, 3), (5, 5), (2, 5), (7, 2), (5, 2), (6, 5)} 
Knight's pos: (0, 1) 
Destination: (1, 5) 


Finding solution...
Solution found!
Objective: 18
Path:
[(0, 1), (2, 2), (4, 3), (2, 4), (0, 5), (1, 3), (3, 2), (4, 0), (2, 1), (0, 2), (2, 1), (4, 2), (6, 3), (7, 5), (5, 4), (6, 6), (4, 5), (5, 7), (3, 6), (1, 5)]
Runtime: 0.006837825999980396


### Heuristics for A*

In [6]:
def manhattan(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])


def heuristic(state):
    coin_pos_set = state.coin_pos_set.copy()
    current_knight_pos = state.knight_pos
    point = 0
    while len(coin_pos_set) > 0:
        min_distance = state.board_size * 2 + 1
        nearest_coin = (0, 0)
        for coin_pos in coin_pos_set:
            distance = manhattan(coin_pos, current_knight_pos)
            if distance < min_distance:
                min_distance = distance
                nearest_coin = coin_pos

        coin_pos_set.remove(nearest_coin)
        current_knight_pos = nearest_coin
        point += (math.ceil((min_distance) / 3) + state.coin_value)

    point += math.ceil(manhattan(state.destination, current_knight_pos) / 3)
    return point


### A*

In [7]:
import heapq

class Node:
    def __init__(self, board):
        self.state = board
        self.parent = None
        self.game_cost = 0
        self.g = 0
        self.h = 0
        self.f = 0

    def __lt__(self, other):
        return self.f < other.f

    def __str__(self):
        return str(self.state)


def child_node(node, action):
    child, gain = node.state.move_knight(action)
    child = Node(child)
    child.parent = node
    return child, gain


def solution(node):
    path = []
    path.append(node.state.knight_pos)
    while node.parent != None:
        node = node.parent
        path.append(node.state.knight_pos)
    path = path[::-1]
    return path


def a_star(problem):
    initial_node = Node(problem.initial_state())
    num_nodes = 1
    fringe = []
    heapq.heappush(fringe, initial_node)
    knight_node = Node(problem.initial_state())

    while len(fringe) != 0:
        current_node = heapq.heappop(fringe)
        num_nodes += 1
        if num_nodes == 400000:
            print('Exited! No solutions found')
        if problem.goal_test(current_node.state):
            path = solution(current_node)
            final_cost = - (current_node.game_cost - current_node.state.goal_value)
            # print("Number of nodes:", num_nodes)
            return path, final_cost, num_nodes
        for new_pos in current_node.state.possible_moves():
            child, point_gained = child_node(current_node, new_pos)[0], child_node(current_node, new_pos)[1]
            # create child node
            child.game_cost = current_node.game_cost + point_gained
            child.g = child.game_cost
            child.h = heuristic(child.state)
            child.f = child.g + child.h
            heapq.heappush(fringe, child)


In [8]:
if __name__ == "__main__":
    board_size = 8  # int(input('Board size: '))
    num_coin = 9  # int(input('Number of coins: '))
    num_obs = 8  # int(input('Number of obstacles: '))
    coin_pos_set = {(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)}
    random.seed(2)
    # problem = Problem(board_size=board_size, num_coin=num_coin, num_obstacles=num_obs, coin_pos_set=coin_pos_set,
    # knight_pos=(7, 5), dest_pos=(5, 4))
    problem = Problem(board_size=board_size, num_coin=num_coin, num_obstacles=num_obs, algorithm='Astar')
    print(
        'Environment:\nBoard_size:', problem.board_size, '\nNumber of coins:', problem.num_coin,
        '\nCoin positions set:',
        problem.coin_pos_set, '\nNumber of obstacle:', problem.num_obstacles, '\nObstacles\' set:',
        problem.obstacle_pos_set,
        '\nKnight\'s pos:', problem.knight_pos,
        '\nDestination:', problem.dest_pos, '\n')

    path, cost, num_nodes = a_star(problem)
    print(path)
    print(cost)
    # print(num_nodes)

    # if path != None:
    #     a = input('Run the demonstration of the path? [y/n] ')
    #     if a == 'y':
    #         show_board(problem.board_size, problem.knight_pos, problem.coin_pos_set, problem.obstacle_pos_set,
    #                    problem.dest_pos, path, problem.board.coin_value, problem.board.normal_value)


Environment:
Board_size: 8 
Number of coins: 9 
Coin positions set: {(6, 6), (2, 2), (5, 7), (7, 5), (0, 5), (4, 3), (2, 4), (0, 2), (4, 0)} 
Number of obstacle: 8 
Obstacles' set: {(2, 7), (6, 7), (3, 3), (5, 5), (2, 5), (7, 2), (5, 2), (6, 5)} 
Knight's pos: (0, 1) 
Destination: (1, 5) 

[(0, 1), (2, 2), (4, 3), (2, 4), (0, 5), (1, 3), (2, 1), (0, 2), (2, 1), (4, 0), (3, 2), (4, 4), (5, 6), (7, 5), (5, 4), (6, 6), (4, 5), (5, 7), (3, 6), (1, 5)]
18


### Backtracking

In [9]:
from collections import deque


class Node:
    def __init__(self, board):
        self.state = board

    def __eq__(self, other):
        if self.state.knight_pos == other.state.knight_pos and self.state.coin_pos_set == other.state.coin_pos_set:
            return True
        return False

    def __repr__(self):
        return str(self.state.knight_pos)


def check_feasibility(problem):
    """
    A function to check if the problem is solvable using BFS
    """
    
    initial_node = Node(problem.initial_state())
    if problem.goal_test(initial_node.state):
        return True

    frontier = deque([])
    frontier.appendleft(initial_node)
    explored = set()

    while len(frontier) != 0:
        current_node = frontier.pop()
        explored.add(current_node.state.knight_pos)

        for new_pos in current_node.state.possible_moves():
            next_state, _ = current_node.state.move_knight(new_pos)
            # create child node
            child_node = Node(next_state)
            if child_node.state.knight_pos not in explored:
                if problem.goal_test(child_node.state):
                    return True
                frontier.appendleft(child_node)

    print("Not solvable.")
    return False


def backtracking(problem):
    initial_node = Node(problem.initial_state())
    initial_node.state.goal_value = 0
    path = []
    path.append(initial_node)
    cost = 0
    min_cost = float('inf')
    best_path = []

    def solution():
        nonlocal cost
        nonlocal min_cost
        nonlocal best_path
        if cost < min_cost:
            min_cost = cost
            best_path = path[:]

    def check(node):
        return node not in path

    def Try():
        nonlocal cost
        current_node = path[-1]
        candidates = []

        for new_pos in current_node.state.possible_moves():
            next_state, point_gained = current_node.state.move_knight(new_pos)
            next_state.goal_value = 0
            # create child node
            child_node = Node(next_state)
            candidates.append((child_node, point_gained))

        candidates.sort(key=lambda x: x[1])

        for candidate in candidates:
            child_node = candidate[0]
            point_gained = candidate[1]
            if check(child_node):
                path.append(child_node)
                cost += point_gained

                if problem.goal_test(child_node.state):
                    solution()
                else:
                    estimated_min_cost = cost + child_node.state.coin_value * len(child_node.state.coin_pos_set)
                    if estimated_min_cost < min_cost:
                        Try()

                path.pop()
                cost -= point_gained
    
    Try()
    max_cost = -min_cost

    print(len(best_path))
    print("Best_path:", best_path)
    print("Max cost:", max_cost)

    return best_path, max_cost

In [10]:
from time import perf_counter


board_size = 5
num_coin = 5
num_obs = 10
random.seed(3)
problem = Problem(board_size=board_size, num_coin=num_coin, num_obstacles=num_obs,algorithm='backtrack')
if check_feasibility(problem):
  start_time = perf_counter()
  best_path, max_cost = backtracking(problem)
  end_time = perf_counter()
  print(end_time - start_time)

  #show_board(problem.board_size, problem.knight_pos, problem.coin_pos_set, problem.obstacle_pos_set,
                        #problem.dest_pos, best_path, problem.board.coin_value, problem.board.normal_value)

11
Best_path: [(1, 4), (2, 2), (0, 3), (1, 1), (0, 3), (2, 4), (0, 3), (2, 2), (3, 4), (2, 2), (4, 1)]
Max cost: 7
0.07638246200002641
